In [ ]:
import json
import os

sprites_path = 'sprites/'
data_json = 'data.json'
data = {}

extracted_json = 'extracted_data.json'
extracted_colors = {}
k = 2

def load_data(fname):
    with open(fname, 'r') as f:
        source = f.read()
        data = json.loads(source)
    return data


if os.path.exists(data_json):
    print('Loading data')
    data = load_data(data_json)
else:
    print('No data')
print('Done')

if os.path.exists(extracted_json):
    print('Loading extracted data')
    extracted_colors = load_data(extracted_json)
else:
    print('No data')
print('Done')

In [ ]:
import numpy as np

names = []
X = []
Y = []

for name, info in data.items():
    colors = extracted_colors[name]
    color = np.mean(np.array(colors)*255, axis=0).astype(np.uint8)
    height, weight, types = info
    for typ in types:
        names.append(name)
        X.append([height, weight, *color])
        Y.append(typ)

X = np.array(X)
Y = np.array(Y)

In [ ]:
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn import preprocessing

r = []

le = preprocessing.LabelEncoder()
le.fit(Y)
print(le.classes_)
Y_transformed = le.transform(Y)

for j in range(len(X[0])):
#     print(X[:,j])
#     print(Y_transformed)
    r.append(scipy.stats.pearsonr(X[:,j], Y_transformed)[0])

print('pearson', r)
important_features = np.absolute(np.array(r)).argsort()[-4:]
print('important', important_features)
X_important = X[:,important_features]

X_scaled = preprocessing.scale(X_important)


krange = range(1, 50)
retries = 10
train_error = [0]*len(krange)
test_error = [0]*len(krange)
for i in range(retries):
    X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y_transformed, test_size=0.2)
    for j, k in enumerate(krange):
        knn = KNeighborsClassifier(n_neighbors=k)
        _ = knn.fit(X_train, Y_train)
        Y_pred = knn.predict(X_train)
        train_error[j] += metrics.accuracy_score(Y_train, Y_pred) * 100 / retries
        Y_pred = knn.predict(X_test)
        test_error[j] += metrics.accuracy_score(Y_test, Y_pred) * 100 / retries

plt.plot(krange, test_error)
plt.plot(krange, train_error)
plt.show()


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y_transformed, test_size=0.2)
knn = KNeighborsClassifier(n_neighbors=30)
knn.fit(X_train, Y_train)
b = names.index('bulbasaur')
c = names.index('charmander')
s = names.index('squirtle')
p = names.index('pikachu')
names_idx = [b, c, s, p]
Y_pred = knn.predict(X_scaled[names_idx])
types_pred = le.inverse_transform(Y_pred)
for name_idx, typ in zip(names_idx, types_pred):
    print(names[name_idx], typ)
